In [ ]:
%load_ext autoreload
%autoreload 2

# Dask

> DaskPool class

In [ ]:
#| default_exp storage.pool.dask

In [ ]:
#| export
from __future__ import annotations
import abc
from configparser import ConfigParser
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union
import logging
import dask.bag as db  # type: ignore
import dask.dataframe as dd  # type: ignore
import pandas as pd  # type: ignore

In [ ]:
#| export
from data_io_nbdev.data.core import ObservationMeta, PoolQuery
from data_io_nbdev.storage.pool.pool import Pool  # type: ignore

In [ ]:
#| export
@dataclass(kw_only=True)
class DaskPool(Pool[pd.DataFrame]):
    """
    The pool Class to be derived from as shared common interfaces and attributes for ParquetPool and AvroPool

    It has with the following features:

        - use Dask dataframe for lazy data processing
        - using dask delayed to parallelize the data processing like sampling,

    Attributes:

        - recipe: a config file for the pool
        - pl_path: the pool path, a Path object to the parquet file for RECORD, to avro file for EPISODE
        - query: a PoolQuery object
        - meta: the meta information for the data collection
        - logger: a logger object
        - dict_logger: a dictionary logger object
    """

    recipe: ConfigParser  # field(default_factory=get_filemeta_config)
    query: PoolQuery  # field(default_factory=PoolQuery)  # search record based on query in arrays is very inefficient
    meta: ObservationMeta  # field(default_factory=ObservationMeta)  # meta information for the data collection
    pl_path: Optional[
        Path
    ] = None  # Path('.')  # path to parquet file for RECORD, to avro file for EPISODE
    # in record file pool, query is mostly ignored for sample, only for checking.
    # list of dask DataFrame with the target vehicle and driver
    logger: Optional[logging.Logger] = None
    dict_logger: Optional[dict] = None

    def __post_init__(self):
        """Parsing the recipe and set the pool path"""
        super().__post_init__()

        # all required specification for the data collection must be available
        assert all(
            key in self.recipe["DEFAULT"]
            for key in [
                "data_folder",
                "recipe_file_name",
                "coll_type",
            ]
        ), f"recipe specification incomplete!"
        self.pl_path = (
            Path(self.recipe["DEFAULT"]["data_folder"])
            / self.recipe["DEFAULT"]["coll_type"]
        )  # coll_type used as part of the path of the parquet storage location,
        # for example, 'data_folder'/'RECORD' or 'data_folder'/'EPISODE'

    def find(self, query: PoolQuery) -> Optional[pd.DataFrame]:
        """
        Find records by `PoolQuery` with

        Args:

            query: a `PoolQuery` object

        return:
            a DataFrame with all records matching query specification
        """

        df = self.get_query(query).compute()
        assert type(df) == pd.DataFrame, f"df is not a pandas DataFrame"
        return df

    @abc.abstractmethod
    def get_query(
        self, query: Optional[PoolQuery] = None
    ) -> Optional[Union[dd.DataFrame, db.Bag]]:
        """
        Get records by `PoolQuery`

        Args:

            query: a `PoolQuery` object

        return:

            a DataFrame with all records in the query time range
        """

        pass

    def _count(self, query: Optional[PoolQuery] = None):
        """
        Count the number of records in the db.

        Args:

            query: a `PoolQuery` object

        Return:

                the number of records in the db
        """
        items = self.get_query(query)  # either a dask dataframe or a dask bag
        if items is None:
            return 0
        else:
            return len(items.compute())

    @abc.abstractmethod
    def sample(
        self,
        size: int,  # required size of samples
        *,
        query: Optional[PoolQuery] = None,  # `PoolQuery` object, query specification
    ) -> pd.DataFrame:
        """
        Sample a batch of data from the pool

        An abstract method to be implemented by the derived class `ParquetPool` and `AvroPool`

        Args:

            size: the number of records to be sampled
            query: a `PoolQuery` object

        Return:
            a Pandas DataFrame
        """

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
show_doc(DaskPool.get_query)

In [ ]:
show_doc(DaskPool.sample)

In [ ]:
show_doc(DaskPool._count)

In [ ]:
show_doc(DaskPool.__post_init__)

In [ ]:
show_doc(DaskPool.find)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()